In [69]:
import numpy as np
import os
import matplotlib.pyplot as plt

from pathlib import Path
from keras.preprocessing.image import load_img, img_to_array, image_dataset_from_directory
from tensorflow.keras.applications import vgg16

In [73]:

# Get base project directory
project_path = Path(os.getcwd()).parent.parent
datapath = (project_path /'data/processed/')

CLASSES = {0 : 'yes', 1 : 'no'}
# Loops through pathlist and reads and resizes images
def read_image(pathlist : list, size : int)-> list:
    data = []
    for path in pathlist:
        image=load_img(path, color_mode='rgb', target_size=(size, size))
        image=img_to_array(image)
        image=image/255.0
        data.append(image)
    return data

# Makes input and label data from folder locations.
# Loops through location "subfolder/CLASSES"
def get_sets(subfolder : str, CLASSES : dict, size : int) -> tuple[list, list]:
    folder_paths = []
    folder_labels = []
    labels = []
    for k,v in CLASSES.items():
        # input datapath generation
        folder_paths += list((datapath / f"2_split_{v}/{subfolder}").rglob("*"))
    # Label data generation
    folder_labels = [0 if x.stem.split('_')[1] == 'yes' else 1 for x in folder_paths]
    # Extract images from datapaths
    img_list = read_image(folder_paths, size)

    return img_list, folder_labels
# Dataset inspect
# Read images to variables
size = 224
X_train, y_train = get_sets('train', CLASSES, size)
X_val, y_val = get_sets('val', CLASSES, size)
X_test, y_test = get_sets('test', CLASSES, size)

In [74]:
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(X_train[0].shape[0], X_train[0].shape[1], 3))

In [75]:
from keras import Model
import keras
from keras.layers import Dense, Flatten, Dropout, Input
import tensorflow as tf

inputs = Input(shape=(X_train[0].shape[0], X_train[0].shape[1], 3))
x = vgg16.preprocess_input(inputs)
x = vgg_conv(x, training=False)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem_4   (None, 224, 224, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_4 (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_9 (Flatten)         (None, 25088)             0         
                                                                 
 dense_14 (Dense)            (None, 256)               6422

In [76]:

metrics = ['accuracy']
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0003)
model.compile(loss="binary_crossentropy", optimizer='RMSprop', metrics=metrics)

callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience=16, restore_best_weights=True)
history = model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val),np.array(y_val)), epochs=20, batch_size=5, verbose=2, callbacks=callbacks)

Epoch 1/20
25/25 - 100s - loss: 762.2869 - accuracy: 0.5360 - val_loss: 3.8749 - val_accuracy: 0.4286 - 100s/epoch - 4s/step
Epoch 2/20
25/25 - 97s - loss: 1.6991 - accuracy: 0.5680 - val_loss: 0.7183 - val_accuracy: 0.5714 - 97s/epoch - 4s/step
Epoch 3/20
25/25 - 95s - loss: 5.1844 - accuracy: 0.5440 - val_loss: 0.6896 - val_accuracy: 0.5714 - 95s/epoch - 4s/step
Epoch 4/20
25/25 - 95s - loss: 0.6845 - accuracy: 0.6320 - val_loss: 0.6874 - val_accuracy: 0.5714 - 95s/epoch - 4s/step
Epoch 5/20
25/25 - 97s - loss: 0.6809 - accuracy: 0.6320 - val_loss: 0.6858 - val_accuracy: 0.5714 - 97s/epoch - 4s/step
Epoch 6/20


KeyboardInterrupt: 

In [ ]:
tf.keras.u